In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sqlite3
# connect database
conn = sqlite3.connect('test.db')
print('Opened database successfully.');
c = conn.cursor()

Opened database successfully.


In [4]:
c.execute('''DROP TABLE IF EXISTS postings ''')
c.execute(''' 
        CREATE TABLE IF NOT EXISTS postings
       (word VARCHAR(100) PRIMARY KEY,
       postinglist_freq_tfidf TEXT );''')

print('Table postings created successfully.');

c.execute('''DROP TABLE IF EXISTS docmag ''')
c.execute(''' 
        CREATE TABLE IF NOT EXISTS docmag
       (docid INT PRIMARY KEY     NOT NULL,
       maxf INT     NOT NULL,
       mag FLOAT);''')
print('Table docmag created successfully.');

conn.commit()

Table postings created successfully.
Table docmag created successfully.


In [5]:

# store data to database
import re
conn = sqlite3.connect('test.db')
print('Opened database successfully.');
c = conn.cursor()

# delete content in database

sql_delete = """ DELETE FROM docmag"""
c.execute(sql_delete)

# Inserting data from doc_mag
with open("/content/drive/My Drive/doc_mag.txt") as file1:
    f11 = file1.readlines()
    # print(f11)
for x in f11:
  
   x.strip()
   x = re.findall(r'\d+\.?\d*', x)
  #  print(x)
   sql = """insert into docmag(docid,maxf,mag) values (?,?,?)"""
   c.execute(sql,(str(x[0]),x[1],x[2]))
   conn.commit()

Opened database successfully.


In [6]:

sql_delete = """ DELETE FROM postings"""
c.execute(sql_delete)
# Inserting data from inv_txt
with open("/content/drive/My Drive/inv_idx.txt") as file2:
    f22 = file2.readlines()

for x in f22:
   y = re.findall(r'\'(.*)\'[^\[]*(\[.*\])', x)
   #print(y[0][0]) 
   sql_1 = "INSERT INTO postings VALUES (?,?)"
   c.execute(sql_1,[y[0][0],y[0][1]])
   conn.commit()

In [7]:
sql_look = """ SELECT * FROM docmag """
c.execute(sql_look)
print(c.fetchone())
sql_look = """ SELECT * FROM postings """
c.execute(sql_look)
print(c.fetchone())
#conn.close()

(4, 14, 2.7653061224489752)
('feb', "[(4, 1, '0.014285714285714285'), (5, 1, '0.033333333333333333'), (1, 1, '0.014285714285714285'), (2, 1, '0.040000000000000008'), (3, 1, '0.066666666666666666')]")


In [8]:
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
import collections
from collections import Counter
import re
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
import ipywidgets as widgets
from IPython.display import display
%xmode Verbose
import math
import itertools
import operator

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def findstring(string,start,end):
  rank = str()
  start = start-1
  end = end-1
  for index in range(start,end):
    rank = rank + string[index]
  return rank

def findvallist(string,start,end):
   value = list()
   #print(start,end)
   for no in range(0,len(start)):
     act_val = findstring(string,start[no],end[no])
     value.append(act_val)
   return value

def onclick(event):
  with result:
    new(keyword.value)

def new(query):
  keyword = query
  #cleaning our search  query

  keyword = str(keyword).lower() #convert into lowercase
  cleantext = BeautifulSoup(keyword).get_text()

  cleantext = re.sub("\d+", " ", cleantext) #remove digits
  cleantext = re.sub('[^A-Za-z0-9]+', ' ', cleantext) #remove special characters
  cleantext = re.sub("\s+", " ", cleantext) #remove extra spaces

  cleantext = word_tokenize(cleantext)
  cleantext = [word for word in cleantext if word not in stopwords.words('english')]
  counter = Counter(cleantext)
  query = dict()
  maxf = Counter(cleantext).most_common(1)[0][1]
  #maxf of our search query

  # (word, [(freq,tfidf), ...])
  docfreq = 0
  for ele in counter:
    tf = counter[ele]/maxf
    query[ele] = tf
    docfreq = docfreq + (tf * tf)
  docfreq = math.sqrt(docfreq)
  #print(docfreq)
  #print(query)
  sql="select * from postings where word in ({seq})".format(
    seq=','.join(['?']*len(cleantext)))

  c.execute(sql, cleantext)
  result = c.fetchall()
  #print(result)
  # (word, [(did,freq,tfidf), ...])
  # (did, (maxf,magnitude squared))
  all_docs_arr = dict()
  docid = list()
  for row in result:
    #print(row)
    row1 = row[1]
    #print(row1)
    docs = findOccurrences(row1,'(')
    for value in docs:
      docid.extend(row1[value+1])
    #print(docid)
    vals = findOccurrences(row1,'.')
    #print('values=',vals)
    ends = findOccurrences(row1,')')
    #print('end=',ends)
    docvalues = findvallist(row1,vals,ends)
    #print(docvalues)
    for x in range(0,len(docvalues)):
      all_docs_arr[docid[x]] = docvalues[x]
  #print(all_docs_arr)
  docids_unique = list(all_docs_arr.keys())
  #print(docids_unique)


  docs_maxf = {}
  for vals in docids_unique:
    sql_look = """ SELECT * FROM docmag where docid = ? """
    c.execute(sql_look, [vals])
    result_docs = c.fetchall()
    #print(result_docs)
    for val in result_docs:
      docs_maxf[str(val[0])] = val[2]
  #print(docs_maxf)

  cosine_similarity = {}
  for docid in docids_unique:
    numerator = 0
    #print(docid)
    for text in cleantext:
      #print(cleantext)
      try:
        tfifd_doc = float(all_docs_arr[docid])
        #print(tfifd_doc)
        tf_query  = float(query[text])
        numerator = numerator + (tfifd_doc*tf_query)
        denominator = docs_maxf.get(docid)*docfreq
      except:
        break
    try:
      denominator = docs_maxf.get(docid)*docfreq
      #print(numerator,denominator)
      cosine_similarity[docid] = (numerator/denominator)
      #print(cosine_similarity)
    except:
      break
  highest_first = sorted(cosine_similarity.items(),key=operator.itemgetter(1),reverse=True)
  #print(highest_first)
  print("Results fetched in docids from highest similar to lowest")
  for rows in highest_first:
    print(rows[0])

Exception reporting mode: Verbose


In [10]:
import ipywidgets as widgets
from IPython.display import display

keyword = widgets.Text(placeholder= 'Enter search query')
search_button = widgets.Button(description='Search')
result = widgets.Output()

display(keyword,search_button,result)

search_button.on_click(onclick)

Text(value='', placeholder='Enter search query')

Button(description='Search', style=ButtonStyle())

Output()